In [1]:
# Load the necessary packages

import pandas as pd
#import io
#import requests

In [2]:
# Prepare for public availability
#url="https://raw.githubusercontent.com/gpkusuma/dtm_processmining/blob/master/Data/ToyData_PM_DTM.csv"
#s=requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
# Read the synthetic dataset
#df = pd.read_csv('/Users/gunturpk/Documents/work/TOY/ToyData_PM_DTM.csv')
df = pd.read_csv("M:\Work\TOY\Toy_2\ToyData_PM_DTM.csv")

In [4]:
df

,Subject_id,Diagnosis,Time
0,1,I21,01/01/2100
1,1,I21,02/01/2100
2,1,I25,31/01/2100
3,2,I21,02/01/2100
4,2,I21,09/01/2100
...,...,...,...
121,49,I50,01/01/2100
122,49,J15,02/01/2100
123,50,I20,01/01/2100
124,50,I25,02/01/2100


In [4]:
# Remove the duplicate activities and put it into a new data frame
# Dataframe df2 will be created

## Select and then uncomment the necessary statement below
removeduplicate = True
#removeduplicate = False

if removeduplicate == True:
    df2 = df.drop_duplicates(subset= {'Subject_id', 'Diagnosis'}, keep='first')
else:
    df2 = df.copy()

In [5]:
# Function definition for creating pairs
# I.S: event log
# F.S: a set of pairs of diagnosis

def create_pairs(df):
    #initialisation
    n = df.shape[0]
    pool_list = []
    rows = 0

    #create pairs of diagnoses for each subject_id
    for pid in df.Subject_id.unique():
        the_list = []
        the_list = df[df['Subject_id']==pid].Diagnosis.tolist()
        the_timelist = []
        the_timelist = df[df['Subject_id']==pid].Time.tolist()

        pair_container = [(pid,
                           (the_list[i]),
                           the_list[(i + 1) % len(the_list)],
                           (the_timelist[i]),
                           the_timelist[(i + 1) % len(the_list)])
                          for i in range(len(the_list)-1)]

        pool_list.extend(res)

        rows = rows + 1

    return pool_list

In [6]:
pairlist = create_pairs(df2)

In [7]:
# Put the pairs into dataframe

labels = ['Subject_id','Antecedent','Subsequent', 'Time1', 'Time2']
dfpairs = pd.DataFrame.from_records(pairlist, columns=labels)

In [8]:
dfpairs

,Subject_id,Antecedent,Subsequent,Time1,Time2
2,3,I21,I25,01/01/2100,02/03/2100
3,4,I21,I25,21/02/2100,14/06/2100
4,5,I21,I25,01/03/2100,15/04/2100
5,6,I21,I25,01/01/2100,02/01/2100
6,6,I25,J18,02/01/2100,03/01/2100
...,...,...,...,...,...
66,48,I50,I20,02/01/2100,03/01/2100
67,48,I20,N30,03/01/2100,04/01/2100
68,49,I50,J15,01/01/2100,02/01/2100
69,50,I20,I25,01/01/2100,02/01/2100


In [9]:
# Get the unique pairs and count the frequency

dfpair_grouped = dfpairs.groupby(['Antecedent', 'Subsequent']).size().reset_index()
dfpair_grouped.columns = ['Antecedent', 'Subsequent', 'Count']
dfpair_grouped.sort_values('Count', ascending=False, inplace=True)
dfpair_grouped.reset_index(drop=True, inplace=True)
dfpair_grouped['Percentage'] = dfpair_grouped['Count']*100/dfpair_grouped['Count'].sum()
dfpair_grouped['Cumsum'] = dfpair_grouped['Percentage'].cumsum()
dfpair_grouped

,Antecedent,Subsequent,Count,Percentage,Cumsum
0,I20,I25,10,14.084507,14.084507
1,I21,I25,10,14.084507,28.169014
2,I25,J18,8,11.267606,39.436620
3,I20,I48,8,11.267606,50.704225
4,I20,I50,7,9.859155,60.563380
5,I25,N30,5,7.042254,67.605634
6,I20,K29,4,5.633803,73.239437
7,K29,I50,4,5.633803,78.873239
8,I50,J15,3,4.225352,83.098592
9,I20,N30,2,2.816901,85.915493


In [10]:
# Add identifier for each pair (PairID)
dfpair_grouped['PairID'] = dfpair_grouped.index

In [11]:
dfpair_grouped

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID
0,I20,I25,10,14.084507,14.084507,0
1,I21,I25,10,14.084507,28.169014,1
2,I25,J18,8,11.267606,39.436620,2
3,I20,I48,8,11.267606,50.704225,3
4,I20,I50,7,9.859155,60.563380,4
5,I25,N30,5,7.042254,67.605634,5
6,I20,K29,4,5.633803,73.239437,6
7,K29,I50,4,5.633803,78.873239,7
8,I50,J15,3,4.225352,83.098592,8
9,I20,N30,2,2.816901,85.915493,9


In [12]:
dfpairs = pd.merge(dfpairs[['Subject_id','Antecedent','Subsequent','Time1','Time2']],
                   dfpair_grouped[['Antecedent','Subsequent','PairID']], on=['Antecedent','Subsequent'])

In [13]:
dfpairs.sort_values(by=['Subject_id'], inplace=True)

In [14]:
dfpairs.reset_index(drop=True, inplace=True)

In [15]:
dfpairs

,Subject_id,Antecedent,Subsequent,Time1,Time2,PairID
0,1,I21,I25,01/01/2100,31/01/2100,1
1,2,I21,I25,02/01/2100,02/06/2100,1
2,3,I21,I25,01/01/2100,02/03/2100,1
3,4,I21,I25,21/02/2100,14/06/2100,1
4,5,I21,I25,01/03/2100,15/04/2100,1
...,...,...,...,...,...,...
66,48,I20,N30,03/01/2100,04/01/2100,9
67,48,I50,I20,02/01/2100,03/01/2100,14
68,49,I50,J15,01/01/2100,02/01/2100,8
69,50,I20,I25,01/01/2100,02/01/2100,0


In [16]:
dfpairs[dfpairs['PairID']==1]

,Subject_id,Antecedent,Subsequent,Time1,Time2,PairID
0,1,I21,I25,01/01/2100,31/01/2100,1
1,2,I21,I25,02/01/2100,02/06/2100,1
2,3,I21,I25,01/01/2100,02/03/2100,1
3,4,I21,I25,21/02/2100,14/06/2100,1
4,5,I21,I25,01/03/2100,15/04/2100,1
5,6,I21,I25,01/01/2100,02/01/2100,1
7,7,I21,I25,01/01/2100,02/01/2100,1
9,8,I21,I25,01/01/2100,02/01/2100,1
11,9,I21,I25,01/01/2100,02/01/2100,1
13,10,I21,I25,01/01/2100,02/01/2100,1


In [17]:
dfpair_grouped

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID
0,I20,I25,10,14.084507,14.084507,0
1,I21,I25,10,14.084507,28.169014,1
2,I25,J18,8,11.267606,39.436620,2
3,I20,I48,8,11.267606,50.704225,3
4,I20,I50,7,9.859155,60.563380,4
5,I25,N30,5,7.042254,67.605634,5
6,I20,K29,4,5.633803,73.239437,6
7,K29,I50,4,5.633803,78.873239,7
8,I50,J15,3,4.225352,83.098592,8
9,I20,N30,2,2.816901,85.915493,9


In [18]:
dfpair_2 = dfpair_grouped.copy()

In [19]:
dfpair_2['Percentage'] = dfpair_2['Count']*100/dfpair_2['Count'].sum()
dfpair_2['Cumsum'] = dfpair_2['Percentage'].cumsum()

In [20]:
dfpair_2 = dfpair_2[['Antecedent','Subsequent','Count','Percentage','Cumsum','PairID']]

In [21]:
dfpair_2[dfpair_2['Antecedent']=='I21']

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID
1,I21,I25,10,14.084507,28.169014,1


In [23]:
import math
from scipy import stats
from scipy.stats import chi2

In [24]:
def find_RR(A, B, C, D):
    
    if B == 0:
        B = 0.5
        A = A + 0.5
        C = C + 0.5
        D = D + 0.5

    if C == 0:
        C = 0.5
        A = A + 0.5
        B = B + 0.5
        D = D + 0.5

    RR = (A / (A+B))/float(C/(C+D))
    return RR

In [25]:
def find_CI(A, B, C, D):

    if B == 0:
        B = 0.5
        A = A + 0.5
        C = C + 0.5
        D = D + 0.5

    if C == 0:
        C = 0.5
        A = A + 0.5
        B = B + 0.5
        D = D + 0.5
    
    #if A != 0 and C != 0:
    r1 = A/float(A+B)
    r2 = C/float(C+D)
    part1 = (1/float(A))*(B/float(A+B))
    part2 = (1/float(C))*(D/float(C+D))
    sq_root = math.sqrt(part1 + part2)
    lci = (r1/r2) - 1.96 * sq_root
    uci = (r1/r2) + 1.96 * sq_root
    return lci, uci        
    #else:
    #    lci = -1
    #    uci = -1
    #    return -1,-1

In [26]:
def find_ChiSquarePValue(A, B, C, D):
    # Version 1
    #total = A + B + C + D
    #part1 = math.pow((A * D) - (B * C),2)
    #part2 = (A+B)*(C+D)*(B+D)*(A+C)
    #chi_sq = (total * part1) / float(part2)
    
    # Version 2
    
    total = A + B + C + D
    r1 = A+B
    r2 = C+D
    c1 = A+C
    c2 = B+D
    
    # Calculate the "expected value" for each entry
    exp_a = (r1*c1)/total
    exp_b = (r1*c2)/total
    exp_c = (r2*c1)/total
    exp_d = (r2*c2)/total
    
    # Calculate the 'chi_square' for each entry then sum up as chi_sq
    chi_a = math.pow((A-exp_a),2)/float(exp_a)
    chi_b = math.pow((B-exp_b),2)/float(exp_b)
    chi_c = math.pow((C-exp_c),2)/float(exp_c)
    chi_d = math.pow((D-exp_d),2)/float(exp_d)
    
    chi_sq = chi_a + chi_b + chi_c + chi_d
    #p_value = 1 - chi2.cdf(chi_sq, 1)
    p_value = chi2.sf(chi_sq, 1)
    return chi_sq, p_value

### Pair Correlation Analysis

In [27]:
df_J2 = dfpair_2
n = len(df_J2.index)
#starttime = time.time() #dt.datetime.now()
print("- Pairwise Statistical Test for Jensen's Exclusion")
print("-- total number of row: ", n)
#print("-- start at: ",time.ctime())
#step = int(5*n/100)  #get 5% of n
cnt=1

A_lst = []
B_lst = []
C_lst = []
D_lst = []
RR_values_lst = []
LCI_lst = []
UCI_lst = []
chisq_lst = []
pval_lst = []

for i in df_J2.index:
    # Get the pairwise codes
    cLeft = df_J2.iloc[i]['Antecedent']
    cRight = df_J2.iloc[i]['Subsequent']
    # Get the number of each group
    A = df_J2[(df_J2['Antecedent'] == cLeft) & (df_J2['Subsequent'] == cRight)].Count.sum()
    B = df_J2[(df_J2['Antecedent'] == cLeft) & (df_J2['Subsequent'] != cRight)].Count.sum()
    C = df_J2[(df_J2['Antecedent'] != cLeft) & (df_J2['Subsequent'] == cRight)].Count.sum()
    D = df_J2[(df_J2['Antecedent'] != cLeft) & (df_J2['Subsequent'] != cRight)].Count.sum()
    # Get RR
    relativeRiskValue = find_RR(A, B, C, D)
    RR_values_lst.append(relativeRiskValue)
    # Get CI
    lci_val, uci_val = find_CI(A, B, C, D)
    LCI_lst.append(lci_val)
    UCI_lst.append(uci_val)
    #Get Chi square and p-value
    chisq_val, pval_val = find_ChiSquarePValue(A, B, C, D)
    chisq_lst.append(chisq_val)
    pval_lst.append(pval_val)
    #Get A, B, C, D
    A_lst.append(A)
    B_lst.append(B)
    C_lst.append(C)
    D_lst.append(D)
    
    #if(step > 0):
    #    if(cnt%step == 0):
    #        print('--- processed = %.2f %% (time since started: %.2f minutes)' %(cnt/n*100,\
    #                    (time.time() - starttime)/60))
    cnt+=1
# Add the statistical test result into the table
df_J2["A"] = A_lst
df_J2["B"] = B_lst
df_J2["C"] = C_lst
df_J2["D"] = D_lst
df_J2["RR"] = RR_values_lst
df_J2["LCI"] = LCI_lst
df_J2["UCI"] = UCI_lst
df_J2["Chi_square"] = chisq_lst
df_J2["p_value"] = pval_lst

#endtime = time.time() #dt.datetime.now()
#print('--- processed = %.2f %% (time since started: %.2f minutes)' %((cnt-1)/n*100,\
#                        (time.time() - starttime)/60))
#print("------ Total time taken: %.2f minutes" %((time.time() - starttime)/60))

- Pairwise Statistical Test for Jensen's Exclusion
-- total number of row:  18


In [28]:
df_J2

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID,A,B,C,D,RR,LCI,UCI,Chi_square,p_value
0,I20,I25,10,14.084507,14.084507,0,10,21,11,29,1.173021,0.456478,1.889563,0.189824,6.630630e-01
1,I21,I25,10,14.084507,28.169014,1,10,0,11,50,5.146245,4.608917,5.683573,27.712724,1.407340e-07
2,I25,J18,8,11.267606,39.436620,2,8,11,0,52,45.050000,42.244543,47.855457,24.675021,6.785800e-07
3,I20,I48,8,11.267606,50.704225,3,8,23,0,40,21.781250,18.966751,24.595749,11.633385,6.477858e-04
4,I20,I50,7,9.859155,60.563380,4,7,24,4,36,2.258065,1.122619,3.393510,2.111261,1.462188e-01
5,I25,N30,5,7.042254,67.605634,5,5,14,2,50,6.842105,5.288705,8.395506,7.905500,4.928472e-03
6,I20,K29,4,5.633803,73.239437,6,4,27,1,39,5.161290,3.020721,7.301860,2.887368,8.927666e-02
7,K29,I50,4,5.633803,78.873239,7,4,0,7,60,8.160000,7.424413,8.895587,23.120760,1.521392e-06
8,I50,J15,3,4.225352,83.098592,8,3,2,2,64,19.800000,18.258959,21.341041,23.043379,1.583870e-06
9,I20,N30,2,2.816901,85.915493,9,2,29,5,35,0.516129,-1.055226,2.087484,0.718921,3.964981e-01


In [29]:
df_J2[df_J2['RR']>1]

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID,A,B,C,D,RR,LCI,UCI,Chi_square,p_value
0,I20,I25,10,14.084507,14.084507,0,10,21,11,29,1.173021,0.456478,1.889563,0.189824,6.630630e-01
1,I21,I25,10,14.084507,28.169014,1,10,0,11,50,5.146245,4.608917,5.683573,27.712724,1.407340e-07
2,I25,J18,8,11.267606,39.436620,2,8,11,0,52,45.050000,42.244543,47.855457,24.675021,6.785800e-07
3,I20,I48,8,11.267606,50.704225,3,8,23,0,40,21.781250,18.966751,24.595749,11.633385,6.477858e-04
4,I20,I50,7,9.859155,60.563380,4,7,24,4,36,2.258065,1.122619,3.393510,2.111261,1.462188e-01
5,I25,N30,5,7.042254,67.605634,5,5,14,2,50,6.842105,5.288705,8.395506,7.905500,4.928472e-03
6,I20,K29,4,5.633803,73.239437,6,4,27,1,39,5.161290,3.020721,7.301860,2.887368,8.927666e-02
7,K29,I50,4,5.633803,78.873239,7,4,0,7,60,8.160000,7.424413,8.895587,23.120760,1.521392e-06
8,I50,J15,3,4.225352,83.098592,8,3,2,2,64,19.800000,18.258959,21.341041,23.043379,1.583870e-06
10,I25,I21,2,2.816901,88.732394,10,2,17,0,52,13.250000,10.257463,16.242537,5.632342,1.763206e-02


# Pair filtering

In [30]:
# Selection by: 1 - RR>1, 2 - p_values, 3 - both RR and p_values, 4 - both RR and corrected p_value, 5 - select all

selection_by = 1

if selection_by == 1:   
    # By the Relative Risk
    df_selected_pairs = df_J2[df_J2['RR'] > 1]
elif selection_by == 2:
    # By the p_values
    df_selected_pairs = df_J2[df_J2['p_value'] < 0.05]
elif selection_by == 3:
    # By both RR>1 and p_values
    df_selected_pairs = df_J2[(df_J2['p_value'] < 0.05)&(df_J2['RR'] > 1)]
elif selection_by == 4:
    # By both RR>1 and corrected p_values
    df_selected_pairs = df_J2[(df_J2['Correction_Result'] == True)&(df_J2['RR'] > 1)]
else:
    df_selected_pairs = df_J2.copy()

In [31]:
df_selected_pairs #.to_csv("M:\Work\TOY\RR_select.csv", index=False, encoding='utf-8')

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID,A,B,C,D,RR,LCI,UCI,Chi_square,p_value
0,I20,I25,10,14.084507,14.084507,0,10,21,11,29,1.173021,0.456478,1.889563,0.189824,6.630630e-01
1,I21,I25,10,14.084507,28.169014,1,10,0,11,50,5.146245,4.608917,5.683573,27.712724,1.407340e-07
2,I25,J18,8,11.267606,39.436620,2,8,11,0,52,45.050000,42.244543,47.855457,24.675021,6.785800e-07
3,I20,I48,8,11.267606,50.704225,3,8,23,0,40,21.781250,18.966751,24.595749,11.633385,6.477858e-04
4,I20,I50,7,9.859155,60.563380,4,7,24,4,36,2.258065,1.122619,3.393510,2.111261,1.462188e-01
5,I25,N30,5,7.042254,67.605634,5,5,14,2,50,6.842105,5.288705,8.395506,7.905500,4.928472e-03
6,I20,K29,4,5.633803,73.239437,6,4,27,1,39,5.161290,3.020721,7.301860,2.887368,8.927666e-02
7,K29,I50,4,5.633803,78.873239,7,4,0,7,60,8.160000,7.424413,8.895587,23.120760,1.521392e-06
8,I50,J15,3,4.225352,83.098592,8,3,2,2,64,19.800000,18.258959,21.341041,23.043379,1.583870e-06
10,I25,I21,2,2.816901,88.732394,10,2,17,0,52,13.250000,10.257463,16.242537,5.632342,1.763206e-02


In [32]:
df_unsig_correlation = df_J2[df_J2['RR'] <= 1]

In [33]:
left='I20'
right='I50'
df_selected_pairs[df_selected_pairs['Antecedent']==left]

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID,A,B,C,D,RR,LCI,UCI,Chi_square,p_value
0,I20,I25,10,14.084507,14.084507,0,10,21,11,29,1.173021,0.456478,1.889563,0.189824,0.663063
3,I20,I48,8,11.267606,50.704225,3,8,23,0,40,21.781250,18.966751,24.595749,11.633385,0.000648
4,I20,I50,7,9.859155,60.563380,4,7,24,4,36,2.258065,1.122619,3.393510,2.111261,0.146219
6,I20,K29,4,5.633803,73.239437,6,4,27,1,39,5.161290,3.020721,7.301860,2.887368,0.089277


In [34]:
df_selected_pairs[df_selected_pairs['Antecedent']==right]

,Antecedent,Subsequent,Count,Percentage,Cumsum,PairID,A,B,C,D,RR,LCI,UCI,Chi_square,p_value
8,I50,J15,3,4.225352,83.098592,8,3,2,2,64,19.8,18.258959,21.341041,23.043379,0.000002
12,I50,M10,1,1.408451,92.957746,12,1,4,2,64,6.6,4.378317,8.821683,3.307427,0.068967
14,I50,I20,1,1.408451,95.774648,14,1,4,0,66,33.5,30.410233,36.589767,13.388571,0.000253


nD1 = df_directed_pairs[(df_directed_pairs['Antecedent']==left)&(df_directed_pairs['Subsequent']==right)].Count.sum()
nD2 = df_directed_pairs[(df_directed_pairs['Antecedent']==right)&(df_directed_pairs['Subsequent']==left)].Count.sum()

nD1=11

nD2=3

stats.binom_test(nD1, n=nD1+nD2, p = 0.5, alternative='greater')

# Directionality

In [36]:
df_directed_pairs = df_selected_pairs.reset_index(drop=True)

In [40]:
# Generating p value using loop with step = 1
# using all pairs data

pval_lst = []
size_lst = []

for i in df_directed_pairs.index:
    # Get the pairwise codes
    D1 = df_directed_pairs.iloc[i]['Antecedent']
    D2 = df_directed_pairs.iloc[i]['Subsequent']
    nD1 = df_directed_pairs[(df_directed_pairs['Antecedent']==D1)&(df_directed_pairs['Subsequent']==D2)].Count.sum()
    nD2 = df_directed_pairs[(df_directed_pairs['Antecedent']==D2)&(df_directed_pairs['Subsequent']==D1)].Count.sum()
    
    if nD1 > nD2:
        p = stats.binom_test(nD1, n=nD1+nD2, p = 0.5, alternative='greater')
        size_lst.append('Greater')
    elif nD1 < nD2:
        p = stats.binom_test(nD1, n=nD1+nD2, p = 0.5, alternative='less')
        size_lst.append('Lower')
    else:
        p = stats.binom_test(nD1, n=nD1+nD2, p = 0.5, alternative='greater')
        size_lst.append('Equal')
    
    pval_lst.append(p)
    
    
    #p = stats.binom_test(nD1, n=nD1+nD2, p = 0.5, alternative='greater')
    #pval_lst.append(p)
    #p2 = stats.binom_test(nD2, n=nD1+nD2, p = 0.5, alternative='greater')
    #print(i, nD1, nD2, p)
    
df_directed_pairs['p_val_direction'] = pval_lst
df_directed_pairs['A_S'] = size_lst

In [41]:
alpha = 0.05

In [42]:
df_directed_pairs['Significantly_greater'] = df_directed_pairs.\
apply(lambda row: (row.p_val_direction < alpha)&(row.A_S == 'Greater') ,axis=1)

In [44]:
df_directed_pairs[['Antecedent','Subsequent','Count','p_val_direction', 'A_S', 'Significantly_greater']]

,Antecedent,Subsequent,Count,p_val_direction,A_S,Significantly_greater
0,I20,I25,10,0.000977,Greater,True
1,I21,I25,10,0.019287,Greater,True
2,I25,J18,8,0.003906,Greater,True
3,I20,I48,8,0.003906,Greater,True
4,I20,I50,7,0.035156,Greater,True
5,I25,N30,5,0.109375,Greater,False
6,I20,K29,4,0.062500,Greater,False
7,K29,I50,4,0.062500,Greater,False
8,I50,J15,3,0.125000,Greater,False
9,I25,I21,2,0.019287,Lower,False


In [183]:
df_directed_pairs_taken = df_directed_pairs[(df_directed_pairs['Significantly_greater']==True)]

In [243]:
df_directed_pairs_taken.to_csv("M:\Work\TOY\Direction_select.csv", index=False, encoding='utf-8')

In [218]:
df_directed_pairs_exclude = df_directed_pairs[(df_directed_pairs['Significantly_greater']==False)]

In [184]:
dfpairs_sig_direction = pd.merge(dfpairs[['id','PairID','Time1','Time2']],
                             df_directed_pairs_taken[['Left','Right','PairID']], on='PairID', how='inner')

In [186]:
pair_Left = dfpairs_sig_direction[['id','Left','Time1','PairID']].rename(columns={"Left": "Diagnosis", "Time1": "Time"})

In [187]:
pair_Right = dfpairs_sig_direction[['id','Right','Time2','PairID']].rename(columns={"Right": "Diagnosis", "Time2": "Time"})

In [199]:
pair_log = pd.concat([pair_Left, pair_Right],ignore_index=True).sort_values(['id','Time'])

In [200]:
pair_log = pair_log.reset_index(drop=True)
pair_log

,id,Diagnosis,Time,PairID
0,1,D,5,14
1,1,B,6,4
2,1,B,6,14
3,1,H,7,4
4,4,J,1,2
...,...,...,...,...
291,98,C,3,27
292,98,G,4,31
293,98,J,5,2
294,98,J,5,31


In [204]:
pair_log['subject_id'] =pair_log['id']

In [205]:
pair_log.to_csv("M:\Work\TOY\ToyPairlog.csv", index=False, encoding='utf-8')

In [202]:
# Merge the overlapping diagnosis

pair_log_merged = pair_log.drop_duplicates(['id','Diagnosis','Time'], keep='first')

In [203]:
pair_log_merged.to_csv("M:\Work\TOY\ToyPairlogMerged.csv", index=False, encoding='utf-8')

***

In [206]:
# Check the re-occurring unwanted pair

In [211]:
pair_log_merged.rename(columns={"Diagnosis": "diag", "Time": "seq"})
#pairlist_merged = create_pairs(pair_log_merged)

C:\Users\scsgk\AppData\Local\conda\conda\envs\my_root\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [214]:
pair_log_merged.rename(columns={"Diagnosis": "diag", "Time": "seq"})

,id,diag,seq,PairID
0,1,D,5,14
1,1,B,6,4
3,1,H,7,4
4,4,J,1,2
5,4,B,2,1
...,...,...,...,...
290,98,F,2,27
291,98,C,3,27
292,98,G,4,31
293,98,J,5,2


In [215]:
pairlist_merged = create_pairs(pair_log_merged.rename(columns={"Diagnosis": "diag", "Time": "seq"}))

In [216]:
# save the pairs into dataframe

labels = ['id','Left','Right', 'Time1', 'Time2']
dfpairs_2nd = pd.DataFrame.from_records(pairlist_merged, columns=labels)

In [220]:
#dfpairs_2nd
dfpairs_2nd = pd.merge(dfpairs_2nd[['id','Left','Right','Time1','Time2']], dfpair_grouped[['Left','Right','PairID']], on=['Left','Right'])

In [221]:
dfpairs_2nd

,id,Left,Right,Time1,Time2,PairID
0,1,D,B,5,6,14
1,8,D,B,5,6,14
2,37,D,B,1,2,14
3,44,D,B,3,4,14
4,63,D,B,2,3,14
...,...,...,...,...,...,...
170,73,G,B,2,3,44
171,81,B,I,2,3,40
172,86,G,C,5,6,24
173,86,I,D,7,8,48


In [222]:
dfpairs_unsig_direction = pd.merge(dfpairs_2nd[['id','PairID','Time1','Time2']],
                             df_directed_pairs_exclude[['Left','Right','PairID']], on='PairID', how='inner')

In [223]:
dfpairs_unsig_direction

,id,PairID,Time1,Time2,Left,Right
0,8,13,4,5,A,D
1,87,13,5,8,A,D
2,58,52,5,8,H,C
3,62,22,4,6,C,G
4,78,22,4,5,C,G
5,94,22,3,6,C,G
6,98,22,3,4,C,G
7,68,15,5,6,E,I
8,69,25,2,4,F,E
9,81,40,2,3,B,I


In [225]:
dfpairs_unsig_correlation = pd.merge(dfpairs_2nd[['id','PairID','Time1','Time2']],
                             df_unsig_correlation[['Left','Right','PairID']], on='PairID', how='inner')

In [226]:
dfpairs_unsig_correlation

,id,PairID,Time1,Time2,Left,Right
0,14,72,4,5,C,F
1,15,68,2,3,J,I
2,24,74,2,3,C,B
3,59,63,2,3,A,J
4,67,69,4,5,C,D
5,73,69,4,7,C,D
6,68,45,2,3,J,F
7,73,44,2,3,G,B


In [227]:
df_selected_pairs_2 = pd.merge(dfpairs_2nd[['id','PairID','Time1','Time2']],
                             df_selected_pairs[['Left','Right','PairID']], on='PairID', how='inner')

In [228]:
df_selected_pairs_2

,id,PairID,Time1,Time2,Left,Right
0,1,14,5,6,D,B
1,8,14,5,6,D,B
2,37,14,1,2,D,B
3,44,14,3,4,D,B
4,63,14,2,3,D,B
...,...,...,...,...,...,...
161,69,25,2,4,F,E
162,81,40,2,3,B,I
163,86,24,5,6,G,C
164,86,48,7,8,I,D


In [230]:
dfpairs_sig_direction_2 = pd.merge(df_selected_pairs_2[['id','PairID','Time1','Time2']],
                             df_directed_pairs_taken[['Left','Right','PairID']], on='PairID', how='inner')

In [231]:
dfpairs_sig_direction_2

,id,PairID,Time1,Time2,Left,Right
0,1,14,5,6,D,B
1,8,14,5,6,D,B
2,37,14,1,2,D,B
3,44,14,3,4,D,B
4,63,14,2,3,D,B
...,...,...,...,...,...,...
148,69,5,1,2,G,F
149,72,5,5,6,G,F
150,78,5,5,6,G,F
151,92,5,2,3,G,F


In [233]:
pair_Left2 = dfpairs_sig_direction_2[['id','Left','Time1','PairID']].rename(columns={"Left": "Diagnosis", "Time1": "Time"})

In [234]:
pair_Right2 = dfpairs_sig_direction_2[['id','Right','Time2','PairID']].rename(columns={"Right": "Diagnosis", "Time2": "Time"})

In [235]:
pair_log_2 = pd.concat([pair_Left2, pair_Right2],ignore_index=True).sort_values(['id','Time'])

In [236]:
pair_log_2 = pair_log_2.reset_index(drop=True)
pair_log_2

,id,Diagnosis,Time,PairID
0,1,D,5,14
1,1,B,6,4
2,1,B,6,14
3,1,H,7,4
4,4,J,1,2
...,...,...,...,...
301,98,C,3,27
302,98,G,4,31
303,98,J,5,2
304,98,J,5,31


In [237]:
pair_log_2['subject_id'] =pair_log_2['id']

In [238]:
pair_log_2.to_csv("M:\Work\TOY\ToyPairlog_2.csv", index=False, encoding='utf-8')

In [239]:
# Merge the overlapping diagnosis

pair_log_merged_2 = pair_log_2.drop_duplicates(['id','Diagnosis','Time'], keep='first')

In [240]:
pair_log_merged_2.to_csv("M:\Work\TOY\ToyPairlogMerged_2.csv", index=False, encoding='utf-8')